<a href="https://colab.research.google.com/github/chunam76/LLM/blob/main/%5B%EC%8B%A4%EC%8A%B5%5D_1_OpenAI_API_%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [실습] OpenAI API 써보기


OpenAI API를 통해 OpenAI의 기능을 호출하고 활용할 수 있습니다.   

[여기](https://platform.openai.com/account/api-keys)를 클릭하여 API 키를 생성할 수 있습니다.   

In [ ]:
!pip install openai tiktoken --upgrade

In [ ]:
!pip show openai
# 버전 확인하는 코드

In [ ]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY']=""

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

client를 통해 openAI의 기능을 사용할 수 있습니다.      

사용 가능한 모델의 목록은 https://platform.openai.com/docs/models 에서 확인 가능합니다.

<br><br><br>
openai의 LLM 모델은 현재 다음의 모델 사용이 가능합니다.

- gpt-4o-2024-08-06 (gpt-4o의 최신 버전)
- gpt-4o (128k, Tokenizer 개선)
<br><br>
- gpt-4o-mini (128k, Tokenizer 개선)
<br><br>
- gpt-4-turbo (128k)

---
- gpt-3.5-turbo (16k Context Window)

- gpt-3.5-turbo-instruct (Legacy)

3.5 instruct 모델을 제외하고, 모든 모델은 Chat 모델에 해당합니다.   
Chat 모델은 채팅 메시지 형태로 데이터를 전달해야 합니다.

#### Message의 구성    

하나의 채팅 메시지는 `role`과 `content` 조합으로 구성됩니다.   
`role`에 따라 system, user, assistant 메시지로 나누어집니다.   

시스템 메시지는 GPT의 행동을 지정합니다.

In [ ]:
system_prompt = '당신은 모든 대화를 반말과 단답형으로만 합니다.'

messages = [
    {'role':'system', 'content': system_prompt},
    # GPT 기본 프롬프트: You are a helpful assistant.


    {'role':'user', 'content':'당신은 누구입니까?'}
    # user message
]

메시지 목록을 전달하여, GPT API를 호출합니다.

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
)

print(response)

In [ ]:
print(response.choices[0].message.content)

temperature, max_tokens 등의 파라미터를 설정할 수 있습니다.

In [ ]:
messages = [
    {'role':'system', 'content':system_prompt},

    {'role':'user', 'content':'당신은 몇 년까지의 데이터로 학습되었나요?'}
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
    temperature=0.2,
    # temperature: 무작위 출력을 조절: (0-2) 0에 가까울수록 정해진 답변을 수행

    max_tokens = 512,
    # 출력 최대 토큰 수 조절: 초과할 경우 자름

    # n = 4
    # 여러 개의 출력 가능

)
print(response)

In [ ]:
# n != 1 이면 여러 개의 결과 생성

for msg in response.choices:
    print(msg.message.content)


<br><br><br>
### tokens

ChatCompletion의 출력 결과를 보면, usage에 토큰 개수가 저장됩니다.   
토큰의 개수는 모델이 사용하는 토크나이저마다 다르며,   
토큰의 길이는 출력 속도/메모리 사용량/API 요금에 영향을 미칩니다.

tiktoken을 이용하면 모델별 토크나이저를 확인하고, 토큰의 개수를 구할 수 있습니다.

In [ ]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo-instruct")
tokenizer

In [ ]:
prompt = 'GPT 모델별 토크나이저를 확인하고, 프롬프트 토큰의 개수를 구할 수 있습니다.'
tokens = tokenizer.encode(prompt)
print(tokens)
print('총 글자 수:',len(prompt))
print('총 토큰 수:',len(tokens))

GPT-4o 모델은 개선된 토크나이저를 지원합니다.

In [ ]:
tokenizer_4o = tiktoken.encoding_for_model("gpt-4o")
tokenizer_4o

In [ ]:
prompt = 'GPT 모델별 토크나이저를 확인하고, 프롬프트 토큰의 개수를 구할 수 있습니다.'

# GPT4o : 줄어든 토큰 수

tokens= tokenizer_4o.encode(prompt)
print(tokens)
print('총 글자 수:',len(prompt))
print('총 토큰 수:',len(tokens))

<br><br><br>
### seed
LLM은 그 특성상 동일한 input prompt가 들어와도 결과가 항상 다르게 출력되는데,   
 seed 파라미터는 이를 조절하기 위해 만들어졌습니다.

* 출력이 길어지면 결과가 달라집니다. (단일 파라미터로 컨트롤하기는 어려운 것 같습니다..)    


In [ ]:
# 프롬프트 준비
messages = [
    {'role':'system', 'content':'당신은 건강한 식단과 식이의 전문가입니다.'},
    {'role':'user', 'content':'건강한 생활 패턴을 유지하는 방법은 무엇입니까?'}
]


In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    temperature =  0.5,
    max_tokens = 500,
    seed = 8291

)
print(response.choices[0].message.content)

In [ ]:
# 같은 코드로 두 번 실행하기 (살짝 달라짐)

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    temperature =  0.5,
    max_tokens = 500,
    seed = 8291

)
print(response.choices[0].message.content)

번역, 코딩 등의 작업도 수행할 수 있습니다.

In [ ]:
# 번역
prompt = '''한국어 문장: 생성형 AI의 능력이 점점 증가하고 있습니다.
영어 문장: '''

# 프롬프트 준비
messages = [
    {'role':'user', 'content':prompt}
]


In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    temperature =  0.2,
    max_tokens = 500

)
print(response.choices[0].message.content)

In [ ]:
# 코딩
prompt = '''# 주어지는 text 문자열에 대해, 공백을 포함한 특수문자를 모두 제거하는 파이썬 코드
text= "2:0→2:3→4:3→4:4→7:4...'대타 김헌곤 결승타' 삼성, KIA 제압하고 8연패 탈출"

'''

messages = [
    {'role':'system', 'content':'코드만 출력하세요.'},
    {'role':'user', 'content':prompt}
]


In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    temperature =  0.2,
    max_tokens = 500

)
print(response.choices[0].message.content)

In [ ]:
# 되는지 확인해보기



다양한 시스템 메시지는 출력의 형식을 크게 변화시킵니다.    
ChatGPT에서는 user 메시지에 포함하는 내용이지만,    
system 메시지에 넣을 경우 더 효과적입니다.

In [ ]:
messages = [
    {'role':'system', 'content' : '''당신은 공감하거나 격려하지 않으며,
상대를 불쾌하게 합니다. 답변은 반말로 하세요.'''},
    {'role':'user', 'content':'오늘 회사 가기 싫어.'}
]

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    temperature = 1.0
)

print(response.choices[0].message.content)

In [ ]:
print(response.choices[0].message.content)

여러 번의 대화를 저장할 수도 있습니다.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

만약 대화를 계속 이어나가고 싶다면 어떻게 하면 될까요?

입력과 그 결과를 받아, 형식을 맞춰서 messages에 저장하면 됩니다.

## 실습) Multi-turn Conversation    
아래의 코드는 반복문을 이용해 연속적인 대화를 수행하는 코드의 일부입니다.    
새로운 입력(`input()`)과 모델의 출력(`ai_msg`)을 이용해  
연속적인 대화를 이어갈 수 있도록 코드를 수정하세요.

In [ ]:
system_prompt = '당신은 모든 것에 부정적입니다. 답변은 반말로 하세요.'
# 다른 프롬프트로 바꾸셔도 됩니다

history = [
  {"role": "system", "content": system_prompt},
]

# 입력을 4번 수행해야 종료됩니다 (ESC로 중간 종료 가능)

for i in range(4):
    usr_msg = input() # 사용자 대화 입력

    print ("User:",usr_msg) # 그대로 출력

    # TODO: history에 형식 맞춰서 usr_msg append 하기

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = history
    )
    # ai msg 생성

    ai_msg = response.choices[0].message.content

    print("AI:", ai_msg) # 그대로 출력

    # TODO: history에 형식 맞춰서 ai_msg append 하기




## Structured Chat (24.08.06)   

GPT-4o의 최신 버전은 구조화된 출력을 제공합니다.

In [ ]:
from pydantic import BaseModel

class DrugDesc(BaseModel):
    성분명: str
    증상: list[str]
    주의사항: list[str]

class DrugInfo(BaseModel):
    약: list[DrugDesc]

completion = client.beta.chat.completions.parse(
    model = "gpt-4o-2024-08-06",
    messages = [
        {'role':'system', 'content':'다음 데이터를 분석하세요. 새로운 내용을 추가하지 마세요.'},

        {"role": "user", "content": """아세트아미노펜은 해열진통제이다.
발열 및 두통, 신경통, 근육통, 월경통, 염좌통 등을 가라앉히는 데 사용된다.
그 외에도 생리통 및 치통, 관절통, 류마티스성 통증 등에도 사용 가능하다.
아세트아미노펜 단일 성분으로 이뤄진 약 외에 감기약과 같은 복합제에도 함유되어 있는
경우가 많으므로 중복 복용하지 않도록 주의가 필요하다. """},

        {"role":'user', 'content':'''미다졸람은 벤조디아제핀 계열에 속하는 약물이다.
뇌에서 억제성 신경전달물질의 작용을 강화시켜 진정효과를 나타내는 약물이다.
효과가 빠르게 나타나고 짧은 시간 동안 효과가 지속된다.
내시경검사나 수술 전에 진정 목적으로 사용된다.
졸음이나 주의력 저하 등의 부작용을 유발할 수 있으므로
투여 후 자동차 운전이나 위험한 기계 조작을 하지 않도록 한다.'''}],

    response_format= DrugInfo,
)

druginfo = completion.choices[0].message.parsed

druginfo

### Embedding

임베딩은 텍스트를 벡터로 변환합니다.   
OpenAI는 3개의 임베딩 모델을 지원합니다.

- text-embedding-3-large
- text-embedding-3-small
- text-embedding-ada-002 (구버전, 기본값)


#### 임베딩의 활용 예시)
- 검색 : 입력 쿼리와 데이터베이스 문장들 간의 관련도 계산하여 순위 매기기
- 추천 : 텍스트의 연관성을 기준으로 추천하기

In [ ]:
emb = client.embeddings.create(
    model = 'text-embedding-3-large',
    input = '삼성SDS, 생성형 AI 사업 본격화…"패브릭스"와 "브리티 코파일럿" 기반 공공 디지털 혁신 지원',
)

In [ ]:
emb

In [ ]:
emb.data[0].embedding[0:3]

방금 넣은 문장의 임베딩을 다른 문장들의 임베딩과 비교해 보겠습니다.   
아래의 코드는 다소 길지만, 실제 어플리케이션에서는 코드 한 두 줄로 표현할 수 있습니다.

In [ ]:
import numpy as np

# 입력 텍스트의 임베딩 생성
query = '삼성SDS, 생성형 AI 사업 본격화…"패브릭스"와 "브리티 코파일럿" 기반 공공 디지털 혁신 지원'

response = client.embeddings.create(
    input=query,
    model="text-embedding-3-large"
)

query_emb = response.data[0].embedding
query_emb = np.array(query_emb).astype("float32") # 계산을 빠르게 하기 위해서

# 대상 텍스트의 임베딩 생성
target_texts = [
    "신한투자-'삼성SDS, 생성형AI 솔루션으로 성장모멘텀…목표가↑'",
    "AI가 年 310조 경제효과 창출…정부, AI 일상화에 7102억 투입",
    "1114회 로또 1등 각 15억원씩…1곳서 수동 5명(종합)",
    "교보증권, 디지털 전환 가속화…생성형 AI 활용 사내교육 실시"
]

# 4개 문장의 임베딩 생성
response_candidates = client.embeddings.create(
    input=target_texts,
    model="text-embedding-3-large"
)


target_embeds = [record.embedding for record in response_candidates.data] # 4개의 임베딩 저장
target_embeds = np.array(target_embeds).astype("float32")

In [ ]:
# 코사인 유사도 계산
def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2.T)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2, axis=1)
    similarity = dot_product / (norm1 * norm2)
    return 1 -similarity

# 유클리드 거리 계산
def euclidean_distance(embedding1, embedding2):
    distances = np.linalg.norm(embedding2 - embedding1, axis=1)
    return distances

# query_emb와 target_embeds의 코사인 거리 계산
cosine_distances = cosine_similarity(query_emb, target_embeds)

# query_emb와 target_embeds의 유클리드 거리 계산
euclidean_distances = euclidean_distance(query_emb, target_embeds)


print('Query:',query)
print('---')


for i, (cosine_distance, euclidean_distance) in enumerate(zip(cosine_distances, euclidean_distances)):
    print(target_texts[i])
    print(f"코사인 거리: {cosine_distance:.4f}")
    print(f"유클리드 거리: {euclidean_distance:.4f}")
    print('---')

문제의 질문에 가장 가까웠던 텍스트는 첫 번째 텍스트라는 것을 확인할 수 있습니다.

## 이미지 생성 (DALL-E 3)
DALL-E 는 OpenAI의 이미지 생성 인공지능입니다.   
prompt에 원하는 그림의 묘사를 넣으면 생성 가능합니다.

`dall-e-2`, `dall-e-3`를 사용 가능합니다.

In [ ]:
# 계정당 8~16회 /1분 제한

response  = client.images.generate(
  model="dall-e-3",
  prompt="""An elephant with a space suit, flowing through the space with stars and planets.
The elephant is smiling and has a happy face.""",
  n=1,
  size="1024x1024"
)
response

response에는 생성된 그림의 링크가 포함되어 있습니다.    

In [ ]:
image_link = response.data[0].url
image_link

revised_prompt는 사용자의 프롬프트를 더 자세하게 수정합니다.   
이는 Dall-E 3 에서 제안한 기술입니다.

In [ ]:
print(response.data[0].revised_prompt)

In [ ]:
from IPython.display import Image
import requests

# 이미지 출력
img = Image(url = image_link)
response = requests.get(image_link)

# 이미지를 파일로 저장
with open('your_image.png', 'wb') as file:
    file.write(response.content)

img


##  이미지 프롬프트 전달하기

이미지 파일을 OpenAI에 전달하여 프롬프트에 추가할 수도 있습니다.   
content에 image_url이나 base64로 불러온 이미지를 전달하면 됩니다.

In [ ]:
# 링크로 이미지 전달하기

messages = [
    {"role": "user", "content": [
        {"type": "text",
                 "text": "이 그림을 묘사하고, 일반적인 그림과 비교해서 특이한 점을 언급하세요."
        },

        {"type": "image_url",
                "image_url": {"url": image_link}
        },
    ]}

]

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    max_tokens = 1024,
)

print(response.choices[0].message.content)

In [ ]:
# 오프라인 이미지 base64로 로드하여 저장하기
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# 이미지 경로
image_path = "your_image.png"
base64_image = encode_image(image_path)

messages = [
    {"role": "user", "content": [
        {"type": "text",
                 "text": "이 그림엔 무엇이 있나요?"
        },

        {"type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        },
    ]}
]


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages= messages,
    max_tokens=1024,
)

print(response.choices[0].message.content)


------

<br><br><br>

# Voice API(음성 API)

OpenAI의 TTS와 Whisper를 사용할 수 있습니다.

## Text-to-speech (텍스트 음성 변환, TTS)    

모델과 목소리(alloy, echo, fable, onyx, nova, shimmer), input을 입력하면 음성 파일을 생성합니다.


In [ ]:
speech_file_path = "./test.mp3"
response = client.audio.speech.create(
  model="tts-1", #"tts-1-hd" : 2x price, HD
  voice="nova",
  input="""LLM은 Large Language Model의 약자입니다. 대용량의 코퍼스를 학습시킨 머신 러닝 모델로,
LLama 3.1, Mistral 2 Large, Grok 2가 최근 출시되었습니다."""
)

response.stream_to_file(speech_file_path)
# 저장

## Speech-to-Text (음성 인식)   

OpenAI의 Whisper는 오디오 파일을 글자로 변환하는 전사(Transcription) 기능을 지원합니다.


pyaudio와 wave를 이용하여 음성을 녹음할 수 있습니다.

-- **코랩에서는 아래의 코드가 실행되지 않으므로, 녹음 대신 위에서 만든 파일을 활용하겠습니다.**

In [ ]:
# 관련 라이브러리 설치
# !pip install pyaudio wave

In [ ]:
# import pyaudio
# import wave

# # 녹음 설정
# FORMAT = pyaudio.paInt16  # 오디오 형식
# CHANNELS = 1  # 모노 오디오
# RATE = 44100  # 샘플링 레이트 (Hz)
# CHUNK = 1024  # 버퍼 크기
# RECORD_SECONDS = 5  # 녹음 시간 (초)
# OUTPUT_FILENAME = "recorded_audio.wav"  # 저장할 파일 이름

# # PyAudio 초기화
# audio = pyaudio.PyAudio()

# # 오디오 스트림 열기
# stream = audio.open(format=FORMAT, channels=CHANNELS,
#                     rate=RATE, input=True,
#                     frames_per_buffer=CHUNK)

# print("녹음 중...")

# frames = []

# # 녹음 데이터 수집
# for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#     data = stream.read(CHUNK)
#     frames.append(data)

# print("녹음 완료!")

# # 오디오 스트림 닫기
# stream.stop_stream()
# stream.close()
# audio.terminate()

# # WAV 파일로 저장
# with wave.open(OUTPUT_FILENAME, 'wb') as wf:
#     wf.setnchannels(CHANNELS)
#     wf.setsampwidth(audio.get_sample_size(FORMAT))
#     wf.setframerate(RATE)
#     wf.writeframes(b''.join(frames))

# print(f"녹음된 오디오가 '{OUTPUT_FILENAME}' 파일로 저장되었습니다.")

녹음된 파일의 경로를 집어넣어, 전사(transcript)를 수행합니다.

In [ ]:
audio_file= open("./test.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model = "whisper-1",
  file = audio_file
  # prompt = 'Llama 3.1, Mistral 2 Large, Groq 2'
)
transcript.text

Transcription API의 결과를 프롬프트에 포함하면    
음성 데이터를 활용한 어플리케이션을 만들 수도 있습니다.